## A1.4 Selección de características

**1. Importa los datos del archivo “Vino Tinto.csv” a tu ambiente de trabajo. Este archivo lo encontrarás en la misma página donde descargaste esta plantilla. Revisa las dimensiones del data frame e imprime en consola tanto dichas dimensiones como las primeras 5 filas de datos**

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importar los datos del archivo A1.3 NASA.csv
data = pd.read_csv('A1.4 Vino Tinto.csv')

# Imprimir las dimensiones del dataframe
print("Dimensiones del dataframe: ", data.shape)

# Imprimir las primeras 15 filas del dataframe
print("\nPrimeras 5 filas:\n",data.head())

Dimensiones del dataframe:  (1599, 12)

Primeras 5 filas:
    acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1

**2. Separa el data frame en datos de entrenamiento y datos de prueba con una proporción 80/20. Es decir, el 80% de los datos se usarán para entrenar el modelo y el resto para validar sus resultados. Asegúrate que la partición sea aleatoria, no es una buena práctica simplemente tomar las primeras observaciones para entrenar y las últimas para probar. Imprime en pantalla las dimensiones de ambos conjuntos de datos. Revisa y asegúrate que la cantidad de observaciones de ambos conjuntos de datos sumen a la cantidad de datos original.**

In [33]:
# Separar los datos en dos conjuntos: uno para entrenamiento y otro para prueba 
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Sumar el número de datos en cada conjunto
print("\nNúmero de datos en cada conjunto:")
print("Entrenamiento: ", len(train))
print("Prueba: ", len(test))
print("Total: ", len(train) + len(test))


Número de datos en cada conjunto:
Entrenamiento:  1279
Prueba:  320
Total:  1599


**3. Genera la metodología de selección hacia adelante e imprime en consola los índices o los nombres de las características seleccionadas. Para realizar este proceso, te recomiendo que utilices la función “SequentialFeatureSelector” de la librería “mlxtend.feature_selection”, en este enlace encontrarás más información sobre la misma. Lo más probable es que cuando hayas descargado Anaconda, esta librería no se haya incluido en la distribución, por lo que deberás instalarla manualmente; al final de las instrucciones de la actividad te indico cómo hacerlo. Aquí te dejo una descripción de los parámetros que te recomiendo usar:**

* **estimator** Un modelo de regresión lineal. Te recomiendo usar la función LinearRegression de la librería sklearn.linear_model.
* **k_features** Se puede seleccionar la cantidad de variables de salida que se desean, pero te recomiendo mejor usar un rango, y que el algoritmo determine el número adecuado. Por ejemplo, puedes definir el parámetro como (2,8), si te interesa que el método seleccione entre 2 y 8 variables.
* **forward** Determina si se hace selección hacia adelante (True) o hacia atrás (False); en este caso queremos hacer selección hacia adelante.
* **scoring** La métrica que se usará para determinar si un modelo es mejor que otro, te recomiendo definirla como ‘r2’ para usar la R cuadrada.
* **cv** Si se desea realizar validación cruzada, y cuántas instancias de la misma. Te recomiendo definir este parámetro como 10.


In [34]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
# Generar elemento X
x_train = train.drop('calidad', axis = 1)
# Generar elemento Y
y_train = train.calidad

# Crear el modelo de regresión lineal
model = LinearRegression()

# Configurar y aplicar el Selector de Características
sfs = SFS(model, k_features=(2, 8), forward=True, scoring='r2', cv=10)              

# Ajustar el selector en los datos de entrenamiento
sfs.fit(x_train, y_train)

# Imprimir los nombres de las características seleccionadas
selected_features = list(sfs.k_feature_names_)
print("Características seleccionadas:", selected_features)

Características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


**4. Entrenar un modelo que solamente contenga las variables seleccionadas, predecir la respuesta en las observaciones de prueba y medir la capacidad de predicción del modelo usando la R cuadrada, imprimiendo dicho valor en consola. Para el primer paso, simplemente necesitas usar la función fit en el modelo de regresión lineal creado previamente, asegurándote de no introducir toda la información de X, sino solo de las variables seleccionadas. Para realizar las predicciones, puedes usar la función predict en los datos de prueba, pero recuerda para dichos datos también seleccionar solo las variables de interés. Para el último paso, te recomiendo usar la función r2_score de sklearn.metrics.**

In [35]:
from sklearn.metrics import r2_score

# Selección de características
x_train_selected = train[selected_features]
x_test_selected = test[selected_features]
y_test = test.calidad

# Construir modelo
model.fit(x_train_selected, y_train)

# Realizar las predicciones sobre los datos de prueba con las mismas variables seleccionadas
y_pred = model.predict(x_test_selected)

# Calcular y imprimir el valor de R^2
r2 = r2_score(y_test, y_pred)
print(f"R^2: {r2}")

R^2: 0.4012628835440285


**5. Realizar un proceso de selección hacia atrás a partir de las variables seleccionadas por el método de selección hacia adelante e imprimir en consola los índices o nombres de las variables seleccionadas. Para realizar este proceso, te recomiendo usar la misma función del paso 3, pero definiendo ahora forward=False. También te recomiendo especificar una menor cantidad de variables posibles, por ejemplo: k_features=(2,5).**

In [36]:
#  Configurar y aplicar el Selector de Características
sfs = SFS(model, k_features=(2, 5), forward=False, scoring='r2', cv=10)              

# Ajustar el selector en los datos de entrenamiento
sfs.fit(x_train_selected, y_train)

#  Imprimir los nombres de las características seleccionadas
selected_features_2 = list(sfs.k_feature_names_)
print("Características seleccionadas:", selected_features_2)


Características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


**6. Repetir el paso 4, pero para un modelo que contenga solamente las variables seleccionadas en el paso 5.**

In [37]:
# Selección de características
x_train_selected_2 = train[selected_features_2]
x_test_selected_2 = test[selected_features_2]
y_test = test.calidad

# Construir modelo
model.fit(x_train_selected_2, y_train)

# Realizar las predicciones sobre los datos de prueba con las mismas variables seleccionadas
y_pred = model.predict(x_test_selected_2)

# Calcular y imprimir el valor de R^2
r2 = r2_score(y_test, y_pred)
print(f"R^2: {r2}")

R^2: 0.3958889666765396


**Imprime en pantalla un breve texto que describa tu opinión sobre la diferencia en R cuadrada medida entre los modelos de los pasos 4 y 6 ¿Cuál modelo consideras que es mejor? ¿Por qué?**

El mejor modelo es el último (Paso 6), debido a que, además de tener un R cuadrada menor (R cuadrada de Paso 6 = 0.396, R cuadrada de Paso 4 = 0.401), también utiliza una menor cantidad de variables, lo cual significaría que su R ajustada sería incluso menor.

**Firma de honor:**  "Doy mi palabra que he realizado esta actividad con integridad académica" - Daniel Matthieu Potes Lesoinne #584678